In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
from colorama import Fore

from sklearn.metrics import mean_absolute_error, mean_squared_error
import math

import warnings  # Supress warnings
warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
np.random.seed(7)


In [ ]:
df = pd.read_csv(r"当地天气状况处理.csv")
df = df.drop(['Unnamed: 0'], axis=1)
df

In [ ]:
# df  = df.rename(columns={'日期1':'date'})
# # del df['日期1']
# df

# 查看时序

In [ ]:
from datetime import datetime, date 

df['date'] = pd.to_datetime(df['date'])
df.head().style.set_properties(subset=['date'], **{'background-color': 'dodgerblue'})

In [ ]:
df1 = df[:24*4*15]
df1

In [ ]:
# To compelte the data, as naive method, we will use ffill
df1 = df[:24*4*15]

f, ax = plt.subplots(nrows=7, ncols=1, figsize=(15, 30))


for i, column in enumerate(df1.drop('date', axis=1).columns):
    sns.lineplot(x=df1['date'], y=df1[column].fillna(method='ffill'), ax=ax[i], color='dodgerblue')
#     ax[i].set_title('Feature: {}'.format(column), fontsize=14)
    ax[i].set_ylabel(ylabel=column, fontsize=14)
                      
    ax[i].set_xlim([date(2018, 1, 1), date(2018, 1, 15)])   

In [ ]:
df = df.sort_values(by='date')

# Check time intervals
df['delta'] = df['date'] - df['date'].shift(1)

df[['date', 'delta']].head()

In [ ]:
df['delta'].sum(), df['delta'].count()

In [ ]:
df = df.drop('delta', axis=1)
df.isna().sum()

# 异常值缺失值

In [ ]:
f, ax = plt.subplots(nrows=2, ncols=1, figsize=(15, 15))

df1['总有功功率（kw）'] = df1['总有功功率（kw）'].replace(np.nan, 180000)
ol = df1[df1['总有功功率（kw）']==180000]['总有功功率（kw）'].copy()
sns.lineplot(x=df1['date'], y=df1['总有功功率（kw）'].fillna(np.inf), ax=ax[0], color='dodgerblue', label='modified')
sns.lineplot(x=df1['date'], y=ol, ax=ax[0], color='red', label='original')

ax[0].set_title('Feature: 总有功功率（kw）', fontsize=14)
ax[0].set_ylabel(ylabel='总有功功率（kw）', fontsize=14)
ax[0].set_xlim([date(2018, 1, 1), date(2018, 1, 15)])

o = df1['最高温度'].copy()
df1['最高温度'] = df1['最高温度'].replace(0, np.nan)

sns.lineplot(x=df1['date'], y=o, ax=ax[1], color='darkorange', label='original')
sns.lineplot(x=df1['date'], y=df1['最高温度'].fillna(np.inf), ax=ax[1], color='dodgerblue', label='modified')
ax[1].set_title('Feature: 最高温度', fontsize=14)
ax[1].set_ylabel(ylabel='最高温度', fontsize=14)
ax[1].set_xlim([date(2018, 1, 1), date(2018, 1, 15)])

In [ ]:
f, ax = plt.subplots(nrows=4, ncols=1, figsize=(15, 12))

sns.lineplot(x=df['date'], y=df['总有功功率（kw）'].fillna(0), ax=ax[0], color='darkorange', label = 'modified')
sns.lineplot(x=df['date'], y=df['总有功功率（kw）'].fillna(np.inf), ax=ax[0], color='dodgerblue', label = 'original')
ax[0].set_title('Fill NaN with 0', fontsize=14)
ax[0].set_ylabel(ylabel='Volume', fontsize=14)

mean_drainage = df['总有功功率（kw）'].mean()
sns.lineplot(x=df['date'], y=df['总有功功率（kw）'].fillna(mean_drainage), ax=ax[1], color='darkorange', label = 'modified')
sns.lineplot(x=df['date'], y=df['总有功功率（kw）'].fillna(np.inf), ax=ax[1], color='dodgerblue', label = 'original')
ax[1].set_title(f'Fill NaN with Mean Value ({mean_drainage:.0f})', fontsize=14)
ax[1].set_ylabel(ylabel='Volume', fontsize=14)

sns.lineplot(x=df['date'], y=df['总有功功率（kw）'].ffill(), ax=ax[2], color='darkorange', label = 'modified')
sns.lineplot(x=df['date'], y=df['总有功功率（kw）'].fillna(np.inf), ax=ax[2], color='dodgerblue', label = 'original')
ax[2].set_title(f'FFill', fontsize=14)
ax[2].set_ylabel(ylabel='Volume', fontsize=14)

sns.lineplot(x=df['date'], y=df['总有功功率（kw）'].interpolate(), ax=ax[3], color='darkorange', label = 'modified')
sns.lineplot(x=df['date'], y=df['总有功功率（kw）'].fillna(np.inf), ax=ax[3], color='dodgerblue', label = 'original')
ax[3].set_title(f'Interpolate', fontsize=14)
ax[3].set_ylabel(ylabel='Volume', fontsize=14)

for i in range(4):
    ax[i].set_xlim([date(2018, 1, 1), date(2018, 1, 15)])
    
plt.tight_layout()
plt.show()

In [ ]:
df['总有功功率（kw）'] = df['总有功功率（kw）'].interpolate()

# 数据平滑与采样

重采样可以提供数据的附加信息。有两种类型的重采样: 
上采样是指增加采样频率(例如从几天到几小时) 
下采样是指降低采样频率(例如，从几天到几周) 
在这个例子中，我们将使用。resample()函数

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=3, sharex=True, figsize=(16,12))

sns.lineplot(df['date'], df['总有功功率（kw）'], color='dodgerblue', ax=ax[0])
ax[0].set_title('总有功功率（kw）Volume', fontsize=14)

resampled_df = df[['date','总有功功率（kw）']].resample('1D', on='date').sum().reset_index(drop=False)
sns.lineplot(resampled_df['date'], resampled_df['总有功功率（kw）'], color='dodgerblue', ax=ax[1])
ax[1].set_title('Weekly 总有功功率（kw）Volume', fontsize=14)

resampled_df = df[['date','总有功功率（kw）']].resample('3D', on='date').sum().reset_index(drop=False)
sns.lineplot(resampled_df['date'], resampled_df['总有功功率（kw）'], color='dodgerblue', ax=ax[2])
ax[2].set_title('Monthly 总有功功率（kw）Volume', fontsize=14)

for i in range(3):
    ax[i].set_xlim([date(2018, 1, 1), date(2018, 1, 14)])


# Stationarity(平稳性检验)

目测:绘制时间序列并检查趋势或季节性 
基本统计:分割时间序列并比较每个分区的平均值和方差 
统计检验:增强的迪基富勒检验

In [ ]:
# A year has 52 weeks (52 weeks * 7 days per week) aporx.
rolling_window = 52
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 6))

sns.lineplot(x=df1['date'], y=df1['总有功功率（kw）'], color='dodgerblue')
sns.lineplot(x=df1['date'], y=df1['总有功功率（kw）'].rolling(rolling_window).mean(), color='black', label='rolling mean')
sns.lineplot(x=df1['date'], y=df1['总有功功率（kw）'].rolling(rolling_window).std(), color='orange', label='rolling std')
ax.set_title('Depth to Groundwater: Non-stationary \nnon-constant mean & non-constant variance', fontsize=14)
ax.set_ylabel(ylabel='总有功功率（kw） Volume', fontsize=14)
ax.set_xlim([date(2018, 1, 1), date(2018, 1, 15)])
plt.tight_layout()
plt.savefig('20180101_0115平稳性.png')
plt.show()

现在，我们将检查每个变量: p值小于0.05 检查ADF统计值与critical_values的比较范围

In [ ]:
from statsmodels.tsa.stattools import adfuller

result = adfuller(df['总有功功率（kw）'].values)
result

In [ ]:
df1

In [ ]:
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 6))

def visualize_adfuller_results(series, title, ax=ax):
    result = adfuller(series)
    significance_level = 0.05
    adf_stat = result[0]
    p_val = result[1]
    crit_val_1 = result[4]['1%']
    crit_val_5 = result[4]['5%']
    crit_val_10 = result[4]['10%']

    if (p_val < significance_level) & ((adf_stat < crit_val_1)):
        linecolor = 'red' 
    elif (p_val < significance_level) & (adf_stat < crit_val_5):
        linecolor = 'orange'
    elif (p_val < significance_level) & (adf_stat < crit_val_10):
        linecolor = 'forestgreen'
    else:
        linecolor = 'purple'
    sns.lineplot(x=df1['date'], y=series, ax=ax, color=linecolor)
    ax.set_title(f'ADF Statistic {adf_stat:0.3f}, p-value: {p_val:0.3f}\nCritical Values 1%: {crit_val_1:0.3f}, 5%: {crit_val_5:0.3f}, 10%: {crit_val_10:0.3f}', fontsize=14)
    ax.set_ylabel(ylabel=title, fontsize=14)

visualize_adfuller_results(df1['总有功功率（kw）'].values, 'Temperature', )

plt.tight_layout()
plt.savefig('总有功功率（kw）平稳性.png')
plt.show()

In [ ]:
ts_diff = np.diff(df['总有功功率（kw）'])
df['总有功功率（kw）_diff_1'] = np.append([0], ts_diff)

f, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 6))
visualize_adfuller_results(df['总有功功率（kw）_diff_1'], 'Differenced (1. Order) \n Depth to Groundwater', ax)

# 特征工程

时序提取

In [ ]:
df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month
df['day'] = pd.DatetimeIndex(df['date']).day
df['day_of_year'] = pd.DatetimeIndex(df['date']).dayofyear
df['week_of_year'] = pd.DatetimeIndex(df['date']).weekofyear
df['quarter'] = pd.DatetimeIndex(df['date']).quarter
df['season'] = df['month'] % 12 // 3 + 1

df[['date', 'year', 'month', 'day', 'day_of_year', 'week_of_year', 'quarter', 'season']].head()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

core_columns =  ['总有功功率（kw）']

for column in core_columns:
    decomp = seasonal_decompose(df[column], period=52, model='additive', extrapolate_trend='freq')
    df[f"{column}_trend"] = decomp.trend
    df[f"{column}_seasonal"] = decomp.seasonal

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=4, sharex=True, figsize=(16,8))

for i, column in enumerate(['总有功功率（kw）', '最低温度']):
    
    res = seasonal_decompose(df1[column], freq=52, model='additive', extrapolate_trend='freq')

    ax[0,i].set_title('Decomposition of {}'.format(column), fontsize=16)
    res.observed.plot(ax=ax[0,i], legend=False, color='red')
    ax[0,i].set_ylabel('Observed', fontsize=14)

    res.trend.plot(ax=ax[1,i], legend=False, color='red')
    ax[1,i].set_ylabel('Trend', fontsize=14)

    res.seasonal.plot(ax=ax[2,i], legend=False, color='red')
    ax[2,i].set_ylabel('Seasonal', fontsize=14)
    
    res.resid.plot(ax=ax[3,i], legend=False, color='red')#dodgerblue
    ax[3,i].set_ylabel('Residual', fontsize=14)
plt.savefig('时间序列分解.png')
plt.show()

# 滞后特征

In [ ]:
weeks_in_month = 4

for column in core_columns:
    df[f'{column}_seasonal_shift_b_2m'] = df[f'{column}_seasonal'].shift(-2 * weeks_in_month)
    df[f'{column}_seasonal_shift_b_1m'] = df[f'{column}_seasonal'].shift(-1 * weeks_in_month)
    df[f'{column}_seasonal_shift_1m'] = df[f'{column}_seasonal'].shift(1 * weeks_in_month)
    df[f'{column}_seasonal_shift_2m'] = df[f'{column}_seasonal'].shift(2 * weeks_in_month)
    df[f'{column}_seasonal_shift_3m'] = df[f'{column}_seasonal'].shift(3 * weeks_in_month)

# 探索性数据分析

In [ ]:
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 8))

shifted_cols = df.columns[:8]
corrmat = df[shifted_cols].corr()

sns.heatmap(corrmat, annot=True, vmin=-1, vmax=1, cmap='YlGnBu')
ax.set_title('Correlation Matrix of Lagged Features', fontsize=16)

plt.xticks(rotation=30) 
plt.tight_layout()
plt.savefig('相关性.png')
plt.show()

# 自相关分析

In [ ]:
from pandas.plotting import autocorrelation_plot
df1['总有功功率（kw）_diff_1']= df1['总有功功率（kw）'] - df1['总有功功率（kw）'].shift(1)
autocorrelation_plot(df1['总有功功率（kw）_diff_1'])
plt.savefig('一阶自相关.png')
plt.show()

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.graphics.tsaplots import plot_pacf,plot_acf
print(sm.tsa.stattools.adfuller(df['总有功功率（kw）']))

In [ ]:
acorr_ljungbox(df['总有功功率（kw）'], lags = [6, 12],boxpierce=True)

In [ ]:
# ACF
acf=plot_acf(df['总有功功率（kw）'])
plt.title("总有功功率（kw）的自相关图")
plt.savefig('ACF.png')
plt.show()

In [ ]:
# PACF
pacf=plot_pacf(df['总有功功率（kw）'])
plt.title("总有功功率（kw）的偏自相关图")
plt.savefig('PACF.png')
plt.show()

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

N_SPLITS = 3

X = df['date']
y = df['总有功功率（kw）']

folds = TimeSeriesSplit(n_splits=N_SPLITS)

In [ ]:
f, ax = plt.subplots(nrows=N_SPLITS, ncols=2, figsize=(16, 9))

for i, (train_index, valid_index) in enumerate(folds.split(X)):
    X_train, X_valid = X[train_index], X[valid_index]
    y_train, y_valid = y[train_index], y[valid_index]

    sns.lineplot(
        x=X_train, 
        y=y_train, 
        ax=ax[i,0], 
        color='dodgerblue', 
        label='train'
    )
    sns.lineplot(
        x=X_train[len(X_train) - len(X_valid):(len(X_train) - len(X_valid) + len(X_valid))], 
        y=y_train[len(X_train) - len(X_valid):(len(X_train) - len(X_valid) + len(X_valid))], 
        ax=ax[i,1], 
        color='dodgerblue', 
        label='train'
    )

    for j in range(2):
        sns.lineplot(x= X_valid, y= y_valid, ax=ax[i, j], color='darkorange', label='validation')
    ax[i, 0].set_title(f"Rolling Window with Adjusting Training Size (Split {i+1})", fontsize=10)
    ax[i, 1].set_title(f"Rolling Window with Constant Training Size (Split {i+1})", fontsize=10)

for i in range(N_SPLITS):
    ax[i, 0].set_xlim([date(2018, 1, 1), date(2018, 1, 14)])
    ax[i, 1].set_xlim([date(2018, 1, 1), date(2018, 6, 30)])
plt.savefig('交叉验证.png')
plt.tight_layout()
plt.show()

# 单变量时间序列模型

In [ ]:
train_size = int(0.85 * len(df))
test_size = len(df) - train_size
# df = df.fillna(0)
univariate_df = df[['date', '总有功功率（kw）']].copy()
univariate_df.columns = ['ds', 'y']

train = univariate_df.iloc[:train_size, :]

x_train, y_train = pd.DataFrame(univariate_df.iloc[:train_size, 0]), pd.DataFrame(univariate_df.iloc[:train_size, 1])
x_valid, y_valid = pd.DataFrame(univariate_df.iloc[train_size:, 0]), pd.DataFrame(univariate_df.iloc[train_size:, 1])

print(len(train), len(x_valid))

# ARIMA

In [ ]:
trend_evaluate = sm.tsa.arma_order_select_ic(y_train, ic=['aic', 'bic'], trend='nc')
print('train AIC', trend_evaluate.aic_min_order)
print('train BIC', trend_evaluate.bic_min_order)

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
import warnings

warnings.filterwarnings('ignore')

# Fit model
model = ARIMA(y_train, order=(4,0,0))
model_fit = model.fit()
y_pred = model_fit.forecast(len(x_valid))

# Calcuate metrics
score_mae = mean_absolute_error(y_valid, y_pred) #[19296 rows x 1 columns]
score_rmse = math.sqrt(mean_squared_error(y_valid, y_pred))

print(Fore.GREEN + 'MAE: {}'.format(score_mae))
print(Fore.GREEN + 'RMSE: {}'.format(score_rmse))
# MAE: 26008.946939568425
# RMSE: 34286.492961318734

In [ ]:
model = ARIMA(y_train, order=(4,0,0))
arima_res=model.fit()
arima_res.summary()

In [ ]:
# f, ax = plt.subplots(1)
# f.set_figheight(6)
# f.set_figwidth(15)

# model_fit.plot_predict(1, 1300, ax=ax)
# sns.lineplot(x=x_valid.index, y=y_valid['y'], ax=ax, color='orange', label='Ground truth') #navajowhite

# ax.set_title(f'Prediction \n MAE: {score_mae:.2f}, RMSE: {score_rmse:.2f}', fontsize=14)
# # ax.set_xlabel(xlabel='Date', fontsize=14)
# ax.set_ylabel(ylabel='总有功功率（kw）', fontsize=14)

# ax.set_ylim(100000, 350392)
# plt.show()

In [ ]:
f, ax = plt.subplots(1)
f.set_figheight(4)
f.set_figwidth(15)

sns.lineplot(x=x_valid.index, y=y_pred, ax=ax, color='blue', label='predicted') #navajowhite
sns.lineplot(x=x_valid.index, y=y_valid['y'], ax=ax, color='orange', label='Ground truth') #navajowhite

ax.set_xlabel(xlabel='Date', fontsize=14)
ax.set_ylabel(ylabel='总有功功率（kw）', fontsize=14)

plt.show()

# LSTM

In [ ]:
from sklearn.preprocessing import MinMaxScaler

data = univariate_df.filter(['y'])
#Convert the dataframe to a numpy array
dataset = data.values

scaler = MinMaxScaler(feature_range=(-1, 0))

scaled_data = scaler.fit_transform(dataset)

In [ ]:
# Defines the rolling window
look_back = 52
# Split into train and test sets
train, test = scaled_data[:train_size-look_back,:], scaled_data[train_size-look_back:,:]

def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(look_back, len(dataset)):
        a = dataset[i-look_back:i, 0]
        X.append(a)
        Y.append(dataset[i, 0])
    return np.array(X), np.array(Y)

x_train, y_train = create_dataset(train, look_back)
x_test, y_test = create_dataset(test, look_back)

# reshape input to be [samples, time steps, features]
x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

print(len(x_train), len(x_test))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

#Build the LSTM model
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

#Train the model
model.fit(x_train, y_train, batch_size=16, epochs=10, validation_data=(x_test, y_test))

model.summary()


In [ ]:
# Lets predict with the model
train_predict = model.predict(x_train)
test_predict = model.predict(x_test)

# invert predictions
train_predict = scaler.inverse_transform(train_predict)
y_train = scaler.inverse_transform([y_train])

test_predict = scaler.inverse_transform(test_predict)
y_test = scaler.inverse_transform([y_test])

# Get the root mean squared error (RMSE) and MAE
score_rmse = np.sqrt(mean_squared_error(y_test[0], test_predict[:,0]))
score_mae = mean_absolute_error(y_test[0], test_predict[:,0])
print(Fore.GREEN + 'RMSE: {}'.format(score_rmse))
from sklearn.metrics import r2_score
print('R2-score:',r2_score(y_test[0], test_predict[:,0]))

In [ ]:
x_train_ticks = univariate_df.head(train_size)['ds']
y_train = univariate_df.head(train_size)['y']
x_test_ticks = univariate_df.tail(test_size)['ds']

# Plot the forecast
f, ax = plt.subplots(1)
f.set_figheight(6)
f.set_figwidth(15)

sns.lineplot(x=x_train_ticks, y=y_train, ax=ax, label='Train Set') #navajowhite
sns.lineplot(x=x_test_ticks, y=test_predict[:,0], ax=ax, color='green', label='Prediction') #navajowhite
sns.lineplot(x=x_test_ticks, y=y_test[0], ax=ax, color='orange', label='Ground truth') #navajowhite

ax.set_title(f'Prediction \n MAE: {score_mae:.2f}, RMSE: {score_rmse:.2f}', fontsize=14)
ax.set_xlabel(xlabel='Date', fontsize=14)
ax.set_ylabel(ylabel='总有功功率（kw）', fontsize=14)

plt.show()

# prophet

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import math
from fbprophet import Prophet

In [ ]:
model = Prophet()
model.fit(x_train)

In [ ]:
# x_valid = model.make_future_dataframe(periods=test_size, freq='w')
# Predict on valid set
y_pred = model.predict(x_valid)

# Calcuate metrics
score_mae = mean_absolute_error(y_valid, y_pred.tail(test_size)['yhat'])
score_rmse = math.sqrt(mean_squared_error(y_valid, y_pred.tail(test_size)['yhat']))

print(Fore.GREEN + 'RMSE: {}'.format(score_rmse))

In [ ]:
# Plot the forecast
f, ax = plt.subplots(1)
f.set_figheight(6)
f.set_figwidth(15)

model.plot(y_pred, ax=ax)
sns.lineplot(x=x_valid['ds'], y=y_valid['y'], ax=ax, color='orange', label='Ground truth') #navajowhite

ax.set_title(f'Prediction \n MAE: {score_mae:.2f}, RMSE: {score_rmse:.2f}', fontsize=14)
ax.set_xlabel(xlabel='Date', fontsize=14)
ax.set_ylabel(ylabel='总有功功率（kw）', fontsize=14)

plt.show()

# 自动时间序列(AutoARIMA)

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm

In [ ]:
model = pm.auto_arima(y_train, start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(model.summary())

In [ ]:
y_pred = model.predict(len(x_valid))

In [ ]:
from sklearn.metrics import r2_score
print('R2-score:',r2_score(y_valid, y_pred))

In [ ]:
model.plot_diagnostics(figsize=(16,8))
plt.show()

# 多元时序预测

In [ ]:
feature_columns = ['最高温度', '最低温度', '白天风力风向', '夜晚风力风向', '天气1', '天气2']
target_column = ['总有功功率（kw）']

train_size = int(0.85 * len(df))

multivariate_df = df[['date'] + target_column + feature_columns].copy()
multivariate_df.columns = ['ds', 'y'] + feature_columns

train = multivariate_df.iloc[:train_size, :]
x_train, y_train = pd.DataFrame(multivariate_df.iloc[:train_size, [0,2,3,4,5,6,7]]), pd.DataFrame(multivariate_df.iloc[:train_size, 1])
x_valid, y_valid = pd.DataFrame(multivariate_df.iloc[train_size:, [0,2,3,4,5,6,7]]), pd.DataFrame(multivariate_df.iloc[train_size:, 1])

train.head()

In [ ]:
train  =multivariate_df.iloc[:train_size, :]
train


# 多元prophet

In [ ]:
from fbprophet import Prophet


# Train the model
model = Prophet()
# model.add_regressor('最高温度')
# model.add_regressor('最低温度')
# model.add_regressor('白天风力风向')
# model.add_regressor('夜晚风力风向')
# model.add_regressor('天气1')
# model.add_regressor('天气2')
# # Fit the model with train set
model.fit(train)

# Predict on valid set
y_pred = model.predict(x_valid)

# Calcuate metrics
score_mae = mean_absolute_error(y_valid, y_pred['yhat'])
score_rmse = math.sqrt(mean_squared_error(y_valid, y_pred['yhat']))

print(Fore.GREEN + 'RMSE: {}'.format(score_rmse))


In [ ]:
from sklearn.metrics import r2_score
print('R2-score:',r2_score(y_valid, y_pred['yhat']))

In [ ]:
# Plot the forecast
f, ax = plt.subplots(1)
f.set_figheight(6)
f.set_figwidth(15)

model.plot(y_pred, ax=ax)
sns.lineplot(x=x_valid['ds'], y=y_valid['y'], ax=ax, color='orange', label='Ground truth') #navajowhite

ax.set_title(f'Prediction \n MAE: {score_mae:.2f}, RMSE: {score_rmse:.2f}', fontsize=14)
ax.set_xlabel(xlabel='Date', fontsize=14)
ax.set_ylabel(ylabel='总有功功率（kw）', fontsize=14)

plt.show()

## LGBMRegressor建模

In [ ]:
model_lgb = lgb.LGBMRegressor(
                learning_rate=0.01,
                max_depth=-1,
                n_estimators=1000,
                    boosting_type='gbdt',
                    random_state=2022,
                    objective='regression',
                    num_leaves = '32',
                    verbose=-1)
lgb_model = model_lgb.fit(x,y_train)     #建模
pred_val_y  = lgb_model.predict(x_val)   #预测